In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu' #use cuda(gpu) if not available use cpu
print (device)
block_size = 8
batch_size = 4

cpu


In [4]:
with open ('wizard_of_oz.txt', 'r', encoding='utf-8') as f: #use 'wizard_of_oz.txt as a text and encode it with utf-8'
    text = f.read() #let script read the text
chars = sorted(set(text)) #sort all used characters in text
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [5]:
string_to_int = { ch:i for i,ch in enumerate(chars) } #make string of characters into intigers (numbers)
int_to_string = { i:ch for i,ch in enumerate(chars) } #make intigers into string of characters
encode = lambda s: [string_to_int[c] for c in s] #encode the characters into the intigers
decode = lambda l: ''.join([int_to_string[i] for i in l]) #decode the intigers into characters

data = torch.tensor(encode(text), dtype=torch.long) #turns encoded text into tensors

In [6]:
n = int(0.8*len(data)) #take only 80% of the data
train_data = data[:n] #use the 80% for training
val_data = data [n:] #use the remaining 20% for validation

In [8]:
x = train_data[:block_size] #input sequence (e.g., 0 to 7 if block_size = 8)
y = train_data[1:block_size+1] #target sequence (shifted by one so it predicts the next character) 

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([1]) target is tensor(1)
when input is tensor([1, 1]) target is tensor(80)
when input is tensor([ 1,  1, 80]) target is tensor(28)
when input is tensor([ 1,  1, 80, 28]) target is tensor(39)
when input is tensor([ 1,  1, 80, 28, 39]) target is tensor(42)
when input is tensor([ 1,  1, 80, 28, 39, 42]) target is tensor(39)
when input is tensor([ 1,  1, 80, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([ 1,  1, 80, 28, 39, 42, 39, 44]) target is tensor(32)
